<a href="https://colab.research.google.com/github/Akua-Amoabea/Diabetes_prediction-using-Machine-learning/blob/main/Diabetes_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv("/content/drive/MyDrive/diabetes_prediction_dataset.csv")

In [6]:
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [7]:
data.describe(include="all")

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000,100000.000000,100000.00000,100000.000000,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,3,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
top,Female,NaN,NaN,NaN,No Info,NaN,NaN,NaN,NaN
freq,58552,NaN,NaN,NaN,35816,NaN,NaN,NaN,NaN
mean,NaN,41.885856,0.07485,0.039420,NaN,27.320767,5.527507,138.058060,0.085000
std,NaN,22.516840,0.26315,0.194593,NaN,6.636783,1.070672,40.708136,0.278883
min,NaN,0.080000,0.00000,0.000000,NaN,10.010000,3.500000,80.000000,0.000000
25%,NaN,24.000000,0.00000,0.000000,NaN,23.630000,4.800000,100.000000,0.000000
50%,NaN,43.000000,0.00000,0.000000,NaN,27.320000,5.800000,140.000000,0.000000
75%,NaN,60.000000,0.00000,0.000000,NaN,29.580000,6.200000,159.000000,0.000000


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [10]:
data.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [11]:
data_1 = pd.get_dummies(data['gender'])

In [12]:
data_1

,Female,Male,Other
0,True,False,False
1,True,False,False
2,False,True,False
3,True,False,False
4,False,True,False
...,...,...,...
99995,True,False,False
99996,True,False,False
99997,False,True,False
99998,True,False,False


In [13]:
data = pd.concat([data, data_1], axis=1).reindex(data.index)

In [14]:
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,Female,Male,Other
0,Female,80.0,0,1,never,25.19,6.6,140,0,True,False,False
1,Female,54.0,0,0,No Info,27.32,6.6,80,0,True,False,False
2,Male,28.0,0,0,never,27.32,5.7,158,0,False,True,False
3,Female,36.0,0,0,current,23.45,5.0,155,0,True,False,False
4,Male,76.0,1,1,current,20.14,4.8,155,0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0,True,False,False
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0,True,False,False
99997,Male,66.0,0,0,former,27.83,5.7,155,0,False,True,False
99998,Female,24.0,0,0,never,35.42,4.0,100,0,True,False,False


In [15]:
data.drop('gender', axis=1, inplace=True)

In [16]:
data['Female'] = data['Female'].map({True: 1, False: 0})
data['Male'] = data['Male'].map({True: 1, False: 0})
data['Other'] = data['Other'].map({True: 1, False: 0})

In [17]:
data.head()

,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,Female,Male,Other
0,80.0,0,1,never,25.19,6.6,140,0,1,0,0
1,54.0,0,0,No Info,27.32,6.6,80,0,1,0,0
2,28.0,0,0,never,27.32,5.7,158,0,0,1,0
3,36.0,0,0,current,23.45,5.0,155,0,1,0,0
4,76.0,1,1,current,20.14,4.8,155,0,0,1,0


In [18]:
data = data.drop(columns=['smoking_history'])

In [19]:
X = data.drop(columns=['diabetes'])
y = data['diabetes']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [30]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# **KNN CLASSIFIER**

In [31]:
# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [32]:
y_pred = knn.predict(X_test_scaled)

In [33]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [34]:
# Print the evaluation results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

Accuracy: 0.9656
Confusion Matrix:
[[4582   36]
 [ 136  246]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      4618
           1       0.87      0.64      0.74       382

    accuracy                           0.97      5000
   macro avg       0.92      0.82      0.86      5000
weighted avg       0.96      0.97      0.96      5000



# **RANDOM** **FOREST**

In [37]:
from sklearn.ensemble import RandomForestClassifier
RFModel = RandomForestClassifier(n_estimators=15, criterion='gini', max_depth=50)
RFModel.fit(X_train_scaled, y_train)
print('DTModel Train Score is : ', RFModel.score(X_train_scaled, y_train))
print('DTModel Test Score is : ', RFModel.score(X_test_scaled, y_test))

DTModel Train Score is :  0.9962947368421052
DTModel Test Score is :  0.9696


In [45]:
y_pred = RFModel.predict(X_test_scaled)

In [46]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [47]:
# Print the evaluation results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

Accuracy: 0.9696
Confusion Matrix:
[[4588   30]
 [ 122  260]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      4618
           1       0.90      0.68      0.77       382

    accuracy                           0.97      5000
   macro avg       0.94      0.84      0.88      5000
weighted avg       0.97      0.97      0.97      5000



# **DECISION TREE**

In [56]:
from sklearn.tree import DecisionTreeClassifier
DTModel = DecisionTreeClassifier(criterion='gini')
DTModel.fit(X_train_scaled,y_train)
print('DTModel Train Score is : ', DTModel.score(X_train_scaled,y_train))
print('DTModel Test Score is : ', DTModel.score(X_test_scaled,y_test))

DTModel Train Score is :  0.9979157894736842
DTModel Test Score is :  0.9554


In [57]:
y_pred = DTModel.predict(X_test_scaled)

In [58]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [59]:
# Print the evaluation results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

Accuracy: 0.9554
Confusion Matrix:
[[4501  117]
 [ 106  276]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      4618
           1       0.70      0.72      0.71       382

    accuracy                           0.96      5000
   macro avg       0.84      0.85      0.84      5000
weighted avg       0.96      0.96      0.96      5000

